In [1]:
import numpy as np
import pandas as pd

In [21]:
class prim_iterations:
    def __init__(self, A_matrix, f_vector):
        f_vector = np.dot(A_matrix.T, f_vector)
        A_matrix = np.dot(A_matrix.T, A_matrix)
        lambdas = np.linalg.eigvals(A_matrix)
        lambdas = np.array(map(np.absolute, lambdas))
        lambdas = np.sort(lambdas)
        self.tau = 2/(lambdas[0] + lambdas[-1])
        print 'tau is ', self.tau
        self.P_matrix = np.eye(A_matrix.shape[0]).astype(float) - (A_matrix * self.tau)
        self.g_vector = f_vector * self.tau
    
    def __call__(self, current_x):
        return np.dot(self.P_matrix, current_x) + self.g_vector

In [22]:
class zeidel:
    def __init__(self, A_matrix, f_vector):
        f_vector = np.dot(A_matrix.T, f_vector)
        A_matrix = np.dot(A_matrix.T, A_matrix)
        self.f_coefs_divided_list = (f_vector / np.diag(A_matrix)).tolist()
        A_divided = (A_matrix.T / np.diag(A_matrix)).T
        A_divided = np.eye(A_divided.shape[0]).astype(float) - A_divided
        self.a_coefs_list = [A_divided[i] for i in range(0, A_divided.shape[0])]
    
    def __call__(self, current_x):
        for cur_f_coef, cur_a_coefs, cur_index in zip(self.f_coefs_divided_list,
                                                     self.a_coefs_list,
                                                     range(0, len(current_x))):
            current_x[cur_index] = cur_f_coef + np.dot(cur_a_coefs, current_x)
        return current_x

In [23]:
def make_report(iterating_functor_class, A_matrix, f_vector, eps, init_x = None,
                max_iter = 1000, vector_norm_function = np.linalg.norm):
    if init_x is None:
        init_x = np.zeros(len(f_vector))
    init_x = np.array(init_x).astype(float)
    A_matrix = np.array(A_matrix).astype(float)
    f_vector = np.array(f_vector).astype(float)
    iterating_functor = iterating_functor_class(A_matrix, f_vector)
    cur_num_iter = 0
    x_last = init_x
    x_prev = init_x
    def satisfies_eps_condition():
        if cur_num_iter == 0:
            return False
        return (vector_norm_function(x_prev - x_last) < eps)
    print 'Initial vector is ', init_x
    vals_for_table = []
    while (not satisfies_eps_condition() and cur_num_iter < max_iter):
        cur_num_iter += 1
        x_prev = np.copy(x_last)
        x_last = iterating_functor(x_last)
        vals_for_table.append([cur_num_iter, vector_norm_function(x_prev - x_last)] + 
                             x_last.flatten().tolist())
    if not satisfies_eps_condition():
        print 'Iterations number too large, greater than ', max_iter
    vals_for_table = np.array(vals_for_table)
    cols_names = ['iteration number', 'last difference'] + map(str, range(1, len(init_x) + 1))
    print pd.DataFrame(vals_for_table, columns = cols_names).to_string(index = False)

In [24]:
A_argument = [[-7, -9, 1, -9],
              [-6, -8, -5, 2],
              [-3, 6, 5, -9],
              [-2, 0, -5, -9]]
f_argument = [29, 42, 11, 75]
eps_argument = 1e-6

In [25]:
for some_cl in [prim_iterations, zeidel]:
    make_report(some_cl, A_argument, f_argument, eps_argument)

tau is  0.0060574815772
Initial vector is  [ 0.  0.  0.  0.]
 iteration number  last difference         1         2         3         4
                1     8.226602e+00 -3.864673 -3.216523 -3.034798 -5.760665
                2     7.517395e+00  0.057696  0.775935 -3.725569 -0.789814
                3     6.929423e+00 -3.412839 -1.894580 -5.429137 -5.882798
                4     6.414346e+00 -0.171315  1.286817 -5.569418 -1.355615
                5     5.951385e+00 -3.263638 -1.095804 -6.653059 -5.715128
                6     5.529213e+00 -0.524725  1.566834 -6.522008 -1.719664
                7     5.140950e+00 -3.249433 -0.547462 -7.273940 -5.457268
                8     4.782077e+00 -0.910112  1.718207 -7.031542 -1.963870
                9     4.449396e+00 -3.293749 -0.140400 -7.590920 -5.180713
               10     4.140471e+00 -1.283697  1.802628 -7.316345 -2.138931
               11     3.853322e+00 -3.360266  0.178820 -7.755217 -4.915006
               12     3.586263e+00 -1.6